В этой задаче вам предлагается научиться по заголовку искать статью в некотором заданном множестве.

In [ ]:
!pip install corus 
!pip install pymorphy2
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [ ]:
import random
import string
import pymorphy2
import scipy.spatial.distance as ds
from corus import load_lenta
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

morph = pymorphy2.MorphAnalyzer()



Для простоты возьмём первые 10000 новостей

In [ ]:
path = 'lenta-ru-news.csv.gz'
corpus = []
requests = []
for i, record in zip(range(10000), load_lenta(path)):
    corpus.append((i, record.text))
    requests.append((i, record.title))

Теперь необходимо реализовать класс, отвечающий за поиск новостей. Возможным подходом будет выбрать какое-нибудь векторное представление текста (bag-of-words, tf-idf, word2vec и т.п.) и метрику расстояния (косинусное, Евклидово, манхэттенское и т.п.), а потом сортировать новости по расстоянию до заголовка. Однако, вы можете реализовывать любые ваши идеи.

In [ ]:
class Database:
    my_punctuation = string.punctuation + "—«»"    
    new_stop_words = stopwords.words("russian") + ["какой", "либо", "быть", "он", "она",
                                                   "ну", "по", "именно", "это", "кто", "кроме",
                                                   "который", "они", "то", "на"]    
    def filter_text(text):
        text = text.lower().replace('\n', ' ')
        for p in string.punctuation:
            text = text.replace(p, ' ')
        text = ' '.join([morph.parse(t)[0].normal_form for t in text.split(' ')])
        text = ' '.join([t for t in text.split(' ') if not t.startswith('@') and not t.startswith("http") and 
                         not t in new_stop_words and len(t)>1])
        return text
    
    def __init__(self, corpus):
        self.corpus = corpus
        self.texts = []
        for elem in self.corpus:
            self.texts.append(filter_text(elem[1]))   
            
    def find(self, request, k=10):
        data = [filter_text(request)] + self.texts 
        vector = TfidfVectorizer()
        vect1 = vector.fit_transform(data)
        final_vectors = vect1.toarray()
        
        map_dist = []   # has a structure [(index: distance)]
        for i in range(1, len(final_vectors)):
            map_dist.append((i - 1, ds.cosine(final_vectors[0], final_vectors[i])))
        
        sorted_data = []
        sorted_data = sorted(map_dist, key = lambda x: x[1])
        
        index = []
        for i in range(k):
            index.append(sorted_data[i][0]) 
        
        """
            Этот метод должен принимать на вход текст заголовка и возвращать
            для него k самых вероятных новости.
            В качестве возвращаемого значения ожидается numpy-массив размера k, 
            содержащий id новостей в порядке уменьшения релевантности
        """
        return np.array(index)

In [ ]:
%%time
database = Database(corpus)

Проверим глазами разумность ранжирования новостей на отдельном примере 

In [ ]:
import numpy as np
request_id, request_text = requests[809]
print(f'For request (id={request_id}): {request_text}')
print('Responses are:')
for i, response_id in enumerate(database.find(request_text)):
    print(f'{i}    id={response_id}\t{corpus[response_id][1]}')

Теперь оценим качество ранжирования по метрике Recall@k 

In [ ]:
def get_recall_at_k(targets, predictions, k):
    targets_mask = np.repeat(np.expand_dims(targets, 1), k, axis=1)
    return (predictions[:, :k] == targets_mask).sum() / len(targets)

In [ ]:
test_size = 256
test_k = 20

In [ ]:
test_requests = random.sample(requests, test_size)

In [ ]:
%%time
targets = np.zeros(test_size, dtype=np.int32)
predictions = np.zeros((test_size, test_k), dtype=np.int32)
for i, (request_id, request_text) in enumerate(test_requests):

    targets[i] = request_id
    predictions[i] = database.find(request_text, k=test_k)

In [ ]:
for k in [1, 3, 5, 10, 20]:
    print(f'Recall@{k}:\t{get_recall_at_k(targets, predictions, k):.3f}')